In [1]:
# Importamos las librerías para manipular datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# leemos el DataSet desde el repositorio. Como es mayor a 25mb lo subimos comprimido
airbnb = pd.read_csv("airbnb_us.zip")


In [3]:
# imprimimos las primeras 5 filas del dataframe para entender mejor como son los datos a manipular
airbnb.head()


,id,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,...,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds,price
0,13418779,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,SF,...,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0,750.0
1,3808709,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,DC,...,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0,115.0
2,12422935,Apartment,Private room,"{TV,""Wireless Internet"",Heating,""Smoke detecto...",2,1.0,Real Bed,strict,True,SF,...,-122.429526,Comfort Suite San Francisco,Noe Valley,3,100.0,https://a0.muscache.com/im/pictures/82509143-4...,94131,1.0,1.0,85.0
3,180792,House,Private room,"{TV,""Cable TV"",""Wireless Internet"",""Pets live ...",2,1.0,Real Bed,moderate,True,SF,...,-122.501095,Cozy Garden Studio - Private Entry,Richmond District,159,99.0,https://a0.muscache.com/im/pictures/0ed6c128-7...,94121,1.0,1.0,120.0
4,2658946,Apartment,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",6,1.5,Real Bed,strict,True,DC,...,-77.031189,Charming 2 bdrm in trendy U/14th streets w/par...,U Street Corridor,13,89.0,NaN,20009,2.0,3.0,200.0


In [4]:
# usamos la funcion .info() para entender el tipo de dato de cada variable, las dimensiones del dataframe y cuántos valores no nulos hay por variable.
airbnb.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19309 entries, 0 to 19308
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      19309 non-null  int64  
 1   property_type           19309 non-null  object 
 2   room_type               19309 non-null  object 
 3   amenities               19309 non-null  object 
 4   accommodates            19309 non-null  int64  
 5   bathrooms               19274 non-null  float64
 6   bed_type                19309 non-null  object 
 7   cancellation_policy     19309 non-null  object 
 8   cleaning_fee            19309 non-null  bool   
 9   city                    19309 non-null  object 
 10  description             19309 non-null  object 
 11  first_review            15355 non-null  object 
 12  host_has_profile_pic    19306 non-null  object 
 13  host_identity_verified  19306 non-null  object 
 14  host_response_rate      15013 non-null

In [5]:
# vemos que hay variables que contienen valores nulos y son irrelevantes para nuestro objetivo.
# para eivtar problemas más adelante y que el codigo corra más holgado, listamos las columnas a eliminar del DF
variables_a_eliminar = [
    'id',
    'name',
    'description',
    'thumbnail_url',
    'host_has_profile_pic',
    'host_identity_verified',
    'host_response_rate',
    'host_since',
    'first_review',
    'review_scores_rating',
    'last_review',
    'zipcode',
    'amenities',
]

In [6]:
# eliminamos estas columnas y modificamos el DF en una misma linea
airbnb.drop(variables_a_eliminar, axis=1, inplace=True)

In [7]:
# analizamos nuevamente el df luego de esta ultima modificacion y vemos que aun tenemos valores nulos en algunas columnas que no queremos eliminar
airbnb.isnull().sum()

property_type             0
room_type                 0
accommodates              0
bathrooms                35
bed_type                  0
cancellation_policy       0
cleaning_fee              0
city                      0
instant_bookable          0
latitude                  0
longitude                 0
neighbourhood          1458
number_of_reviews         0
bedrooms                 17
beds                     24
price                     0
dtype: int64

In [8]:
# para la dimension neighbourhood, tenemos 1458 registros nulos de 19.308, es decir el 7,5% de los datos.
# decidimos conservar los registros, completando los nulos en 'neighbourhood' con el valor más repetido para la correspondiente 'city':
# para lograrlo, agrupamos por ciudad y obtenemos el neighbourhood más frecuente en esa city con una funcion lambda (si hubira un empate, .iloc[0] elige el primer neighbourhood)
moda_city = (
    airbnb.groupby('city')['neighbourhood'].agg(lambda x: x.mode().iloc[0]))

In [9]:
# ahora creamos un diccionario con indice city y sus neighbourhood más frecuentes. luego completaremos los nulos recorriendo este diccionario
moda_dict = moda_city.to_dict()
# ahora recorremos la columna neighbourhood. Si encuentra un nulo, remplaza con el diccionario. si no es nulo, deja el valor original
airbnb['neighbourhood'] = airbnb.apply(
    lambda row: moda_dict[row['city']] if pd.isna(row['neighbourhood']) else row['neighbourhood'],
    axis=1
)

In [10]:
# al ser pocos registros y todas variables numericas, vamos a completar el dataframe con la mediana de cada variable
for col in ['bathrooms','bedrooms','beds']:
  airbnb[col] = airbnb[col].fillna(airbnb[col].median())

# aun hay que seguir tranformando el dataframe
# la columna instant_bookable debería ser un booleano donde f=False y t=True
airbnb['instant_bookable'].head()

0    f
1    t
2    t
3    f
4    t
Name: instant_bookable, dtype: object

In [11]:
# para evitar problemas, vamos a modificar el tipo de esta columna y tambien a declarar algunas variables como tipo 'string' ya que python no está pudiendo identificarlas correctamente y las cataloga como 'object'.
airbnb[['property_type','room_type','bed_type','cancellation_policy','city', 'neighbourhood']]=airbnb[['property_type','room_type','bed_type','cancellation_policy','city', 'neighbourhood']].astype('string')
airbnb['instant_bookable']=airbnb['instant_bookable'].map({'f': False , 't': True})
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19309 entries, 0 to 19308
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   property_type        19309 non-null  string 
 1   room_type            19309 non-null  string 
 2   accommodates         19309 non-null  int64  
 3   bathrooms            19309 non-null  float64
 4   bed_type             19309 non-null  string 
 5   cancellation_policy  19309 non-null  string 
 6   cleaning_fee         19309 non-null  bool   
 7   city                 19309 non-null  string 
 8   instant_bookable     19309 non-null  bool   
 9   latitude             19309 non-null  float64
 10  longitude            19309 non-null  float64
 11  neighbourhood        19309 non-null  string 
 12  number_of_reviews    19309 non-null  int64  
 13  bedrooms             19309 non-null  float64
 14  beds                 19309 non-null  float64
 15  price                19309 non-null 

In [12]:
# para las variables categoricas relevantes para el posterior analisis, habría que generar variables dummies numericas para poder considerar estas dimensiones en el modelo de machine learning
city_dummies = pd.get_dummies(airbnb['city'])
property_dummies= pd.get_dummies(airbnb['property_type'])
room_dummies= pd.get_dummies(airbnb['room_type'])
bed_dummies= pd.get_dummies(airbnb['bed_type'])
cancellation_dummies= pd.get_dummies(airbnb['cancellation_policy'])
neigh_dummies = pd.get_dummies(airbnb['neighbourhood'])

In [13]:
# ahora las unimos al DF a trabajar
airbnb = pd.concat([airbnb, city_dummies, property_dummies, room_dummies, bed_dummies, cancellation_dummies, neigh_dummies], axis=1)
# falta eliminar las columnas categoricas originales del DF para que solo queden las dummies
airbnb = airbnb.drop(
    ['city', 'property_type', 'room_type', 'bed_type', 'cancellation_policy', 'neighbourhood'], axis=1)

In [14]:
# guardamos el dataframe procesado como airbnb_df para poder entrenar a los modelos de ML
airbnb.to_csv("airbnb_df.csv", index=False)